In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
"""

In [3]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

In [4]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Nourhen2001/SummarizeReviews_model_v2", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize customer reviews.",  # instruction
        "Product: Bluetooth Headphones - SoundBeats | Positive: 'Excellent sound quality at a great price.', 'Fast delivery and helpful customer support during setup.', 'Great value for money.' | Negative: 'Could be cheaper for the build quality.', 'Delivery was delayed a few days, but customer service was responsive.",  # input
        "" # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1000)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize customer reviews.

### Input:
Product: Bluetooth Headphones - SoundBeats | Positive: 'Excellent sound quality at a great price.', 'Fast delivery and helpful customer support during setup.', 'Great value for money.' | Negative: 'Could be cheaper for the build quality.', 'Delivery was delayed a few days, but customer service was responsive.

### Response:
SoundBeats headphones offer great value for money, with excellent sound quality and fast delivery. However, some customers feel the build could be better for the price.<|eot_id|>


RECOMMANDATION


In [8]:
from unsloth import FastLanguageModel

# Load a model for recommendations
rec_model, rec_tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.2",  # Small & fast for recommendations
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)


==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.50.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [9]:

FastLanguageModel.for_inference(rec_model)
# Define a function to generate recommendations
def generate_recommendations(summary):
    recommendation_prompt = """I will give you customer feedback and you respond with 3 business recommendations.

Example 1:
Feedback: "Great battery life but uncomfortable fit"
Recommendations:
1. Redesign ear cushions for better comfort while maintaining battery
2. Offer multiple ear tip sizes in the package
3. Market the battery life as a key differentiator

Example 2:
Feedback: "Fast shipping but product arrived damaged"
Recommendations:
1. Improve packaging materials to prevent damage
2. Implement quality checks before shipping
3. Create a faster replacement policy for damaged items

Now generate recommendations for:
{}""".format(summary)  # Now summary is correctly passed

    inputs = rec_tokenizer([recommendation_prompt], return_tensors="pt").to("cuda")
    outputs = rec_model.generate(**inputs, max_new_tokens=256)

    return rec_tokenizer.decode(outputs[0], skip_special_tokens=True)


In [10]:

# Example usage
summary = "SoundBeats headphones offer excellent sound quality at a reasonable price, with fast delivery and helpful customer support. However, some feel the build quality could be better for the price, and delivery was delayed for a few days."
recommendations = generate_recommendations(summary)
print(recommendations)

I will give you customer feedback and you respond with 3 business recommendations.

Example 1:
Feedback: "Great battery life but uncomfortable fit"
Recommendations:
1. Redesign ear cushions for better comfort while maintaining battery
2. Offer multiple ear tip sizes in the package
3. Market the battery life as a key differentiator

Example 2:
Feedback: "Fast shipping but product arrived damaged"
Recommendations:
1. Improve packaging materials to prevent damage
2. Implement quality checks before shipping
3. Create a faster replacement policy for damaged items

Now generate recommendations for:
SoundBeats headphones offer excellent sound quality at a reasonable price, with fast delivery and helpful customer support. However, some feel the build quality could be better for the price, and delivery was delayed for a few days.

Recommendations:
1. Upgrade materials used in the headphone construction to improve perceived value
2. Implement more robust shipping methods to ensure timely deliver

In [20]:
!git lfs install
!git clone https://huggingface.co/spaces/tessou/MODELS1




Git LFS initialized.
fatal: destination path 'MODELS1' already exists and is not an empty directory.


In [13]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.3 MB/s eta 0:00:00


In [21]:
save_directory = "/content/my_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('/content/my_model/tokenizer_config.json',
 '/content/my_model/special_tokens_map.json',
 '/content/my_model/tokenizer.json')

In [22]:
!zip -r my_model.zip /content/my_model


  adding: content/my_model/ (stored 0%)
  adding: content/my_model/adapter_model.safetensors (deflated 8%)
  adding: content/my_model/README.md (deflated 66%)
  adding: content/my_model/adapter_config.json (deflated 56%)
  adding: content/my_model/special_tokens_map.json (deflated 71%)
  adding: content/my_model/tokenizer.json (deflated 85%)
  adding: content/my_model/tokenizer_config.json (deflated 94%)


In [28]:
import os
print(os.listdir("/content/"))  # Liste les fichiers à la racine de Colab


['.config', 'my_model.zip', 'huggingface_tokenizers_cache', 'unsloth_compiled_cache', 'MODELS1', 'my_model', 'sample_data']


In [38]:
pip install --upgrade transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 66.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.2
    Uninstalling transformers-4.50.2:
      Successfully uninstalled transformers-4.50.2


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from unsloth import FastLanguageModel
import torch
model_path = "/content/my_model"

# Charger le modèle localement
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=1024,
    dtype=torch.float16,  # Ou bfloat16 selon la compatibilité de ton GPU
    load_in_4bit=True,    # Utiliser 4 bits
    device_map="balanced"  # Ce paramètre aide à gérer la mémoire
)


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
!nvidia-smi


Wed Apr  2 23:57:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P0             26W /   70W |    5786MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import torch
torch.cuda.empty_cache()  # Libère la mémoire GPU inutilisée
